# Purpose: To Test the Sci-kit Image Thresholds for Victoria's Mglur5 Images

Date Created: April 13th, 2021

Dates Edited: April 13th, 2021 - Created this document from the Phuong BeV analysis

June 15th, 2022 - Edited for the New Mglurt Images for Publication

*Step 1: Import necessary packages*

In [ ]:
#import operating system
import os

# import major packages
import numpy as np
import matplotlib.pyplot as plt
import skimage
import PIL as Image

In [ ]:
from skimage.morphology import disk
from skimage.segmentation import watershed
from skimage import data
from skimage.filters import rank
from skimage.util import img_as_ubyte
from skimage.segmentation import clear_border

from scipy import ndimage as ndi

Separating the two imports so that it is a cleaner workflow

In [ ]:
# import specific package functions
from skimage import io
from skimage.filters import try_all_threshold
from skimage.filters import threshold_isodata
from skimage.filters import threshold_li
from skimage.filters import threshold_mean
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from skimage.filters import threshold_triangle
from skimage.filters import threshold_yen

from skimage import morphology
from scipy import ndimage
from skimage.measure import label

## Mglur5 DataSet

*Step 2: Import image for testing thresholds*

In [ ]:
#Defining the local folder location for all files
im_folder_location = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/60x_tiff'

In [ ]:
def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

In [ ]:
#Getting a list of all the seprate animal model folders
arr = os.listdir(im_folder_location)
file_list = np.asarray(arr)

#Since I am working on a mac operating system, this removes the DS_store file
file_list = file_list[1:]
file_list = np.delete(file_list, np.where(file_list == '.DS_Store'))

In [ ]:
#Reading in an example image and checking shape
im = io.imread(str(im_folder_location + '/' + file_list[0]))
im.shape

In [ ]:
#max intensity projection
im_max= np.max(im, axis=0)
im_max.shape

In [ ]:
#Testing which channel the iba stain is on
green_im = im_max[1,:,:]
plt.imshow(green_im, cmap='Greys_r')

*Step 3: Trying multiple tresholds at once and saving the composite image*

In [ ]:
#original with gaussian
j = 0 #initialize a counter
for images in file_list:
    im_name = str(im_folder_location + '/' + file_list[j])
    im = io.imread(im_name)
    im_max= np.max(im, axis=0)
    microglia_im = im_max[1,:,:]
    microglia_im = skimage.filters.gaussian(microglia_im, sigma=0.25)
    fig, ax = try_all_threshold(microglia_im, figsize=(10, 8), verbose=False)

    
    j += 1
    fig_name = str(im_name[:-4] + '_all_thresh.tif')
    fig.savefig(fig_name)
    plt.close('all')
    print(j)

*Move all images with try_all_thresh into their own folder*

In [ ]:
import shutil
import os

source_dir = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/60x_tiff'
target_dir = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/60x_tiff_try_all_thresh'

arr = os.listdir(source_dir)
file_list = np.asarray(arr)

file_list = [x for x in file_list if "all_thresh" in x]

    
for file_name in file_list:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

*Step 4: Based on visual inspection of the try all thresholds, Li threshold does the best*

To determine the size of the small object we should remove we took the size of microglia which are approximately 1600 um^2 according to (https://jneuroinflammation.biomedcentral.com/articles/10.1186/s12974-014-0182-7)

The confocal settings for the OGD severity study provided a 0.575 pixel width = 1 um conversion. To ensure we didn't cut off any potential microglia, we chose a cutoff on the small end to be 1/2 of the average which converts to 265 pixels^2 used in the min_size of the remove small objects function from sci-kit image.

In [ ]:
#mean threshold
k = 0
source_dir = '/Users/hhelmbre/Desktop/Mglur5_publication_analysis/'

#Getting a list of all the seprate animal model folders
arr = os.listdir(im_folder_location)
file_list = np.asarray(arr)

#Since I am working on a mac operating system, this removes the DS_store file
file_list = np.delete(file_list, np.where(file_list == '.DS_Store'))

for images in file_list:
    im_name = str(im_folder_location + '/' + file_list[k])
    im = io.imread(im_name)
    im_max= np.max(im, axis=0)
    microglia_im = im_max[1,:,:]
    microglia_im = skimage.filters.gaussian(microglia_im, sigma=0.25)
    thresh_mean = skimage.filters.threshold_mean(microglia_im)
    binary_mean = microglia_im > thresh_mean
    new_binary_mean = morphology.remove_small_objects(binary_mean, min_size=265)
    new_binary_mean = ndimage.binary_fill_holes(new_binary_mean)
    new_binary_mean = clear_border(new_binary_mean)
    
    
    short_im_name = im_name.split('/')[6]

    np.save(str(source_dir + 'mean_thresh_array/' + short_im_name[:-4] + '_mean_thresh'), new_binary_mean)
    plt.imsave(str(source_dir + 'mean_thresh_image/' + short_im_name[:-4] + '_mean_thresh.png'), new_binary_mean)
    
    print(k)
    k += 1

# Show Package and Software Info

In [ ]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,wget

%watermark -u -n -t -z